<a href="https://colab.research.google.com/github/NID123-CH/DSR-40/blob/main/LLM_AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Specify the LLM model we'll be using
model_name = "microsoft/Phi-3-mini-4k-instruct"
# Configure for GPU usage
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
# Load the tokenizer for the chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create a pipeline object for easy text generation with the LLM
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

**Importing libraries:** We bring in required modules from the hugging face (hf) transformers library to work with the language model.

**Specifing LLM**: We specify the hf repository microsoft/Phi-3-mini-4k-instruct to load the instruct LLM from.

**Loading model and tokenizer**: We load the LLM and its associated tokenizer, which prepares them for use.

**Creating pipeline** : We set up a hf pipeline object to streamline sending prompts and getting responses from the LLM.

In [3]:
# Parameters to control LLM's response generation
generation_args = {
    "max_new_tokens": 512,     # Maximum length of the response
    "return_full_text": False,      # Only return the generated text
}

This code defines settings to manage the LLM’s text generation behavior.

max_new_tokens: limits the response length.
return_full_text: ensures we only get the newly generated part of the response.

**Builing the query function**

In [4]:
def query(messages):
  """Sends a conversation history to the AI assistant and returns the answer.

  Args:
    messages (list): A list of dictionaries, each with "role" and "content" keys.

  Returns:
    str: The answer from the AI assistant.
  """

  output = pipe(messages, **generation_args)
  return output[0]['generated_text']

This function bundles the process of sending a question (within a conversational context) to the LLM and getting the response back.

Input: It takes a list of messages that represent the conversation so far.
Output: It returns the text response generated by the LLM.

**Example usage of the query function**

In [5]:
# Example: Math Problem
messages = [
    {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."},
    {"role": "user", "content": "What about solving the equation 2x + 3 = 7?"}
]
result = query(messages)
print(result)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


 To solve the equation 2x + 3 = 7, follow these steps:

1. Subtract 3 from both sides of the equation:
   2x + 3 - 3 = 7 - 3
   2x = 4

2. Divide both sides by 2:
   2x/2 = 4/2
   x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


These examples show how to use query function.

Context: Each example starts with a “system” message to give the LLM instructions, ensuring it understands its role.

Query: Following which we add the query we want the LLM to work on in a “user” message.

**Builing the chat function**

In [6]:
def chat():
  """Enables interactive chat sessions with the AI assistant."""

  # Initialize the conversation with instructions for the AI assistant
  conversation_history = [
      {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."}
  ]

  # Main chat loop
  while True:
    user_input = input("You: ")

    # Check if the user wants to exit the chat
    if user_input.lower() == "exit":
        break

    # Add user's message to the conversation history
    conversation_history.append({"role": "user", "content": user_input})

    # Get a response from the AI assistant
    try:
        response = query(conversation_history)
        print("Assistant: ", response)

        # Record the AI assistant's response in the conversation history
        conversation_history.append({"role": "assistant", "content": response})

    except Exception as e:
        print(f"An error occurred: {e}, please try again.")

This function creates a chat loop, allowing you to converse with the AI Assistant in real-time.

Interactive chat: The chat continues between user and assistant, until user types “exit”.
Conversation history: It maintains a history of the chat, providing the LLM with context for previous messages.

In [7]:
chat()

You: HII
Assistant:   Hello! How can I help you today?
You: CAN YOU GIVE ME HEALTHCARE ADVICE .I AM 35 YEAR OLD?
Assistant:   Absolutely, I'd be glad to offer some general healthcare advice for a 35-year-old. It's important to remember that individual health needs can vary greatly, so it's always best to consult with a healthcare professional for personalized advice. Here are some general tips:


1. **Regular Check-ups**: Schedule annual physical exams with your primary care physician. These exams can help catch potential health issues early on.


2. **Dental Care**: Maintain good oral hygiene by brushing twice a day, flossing daily, and visiting your dentist for regular cleanings and check-ups.


3. **Diet and Nutrition**: Eat a balanced diet rich in fruits, vegetables, whole grains, lean proteins, and healthy fats. Limit processed foods, sugar, and saturated fats.


4. **Exercise**: Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous activity each week

In [8]:
chat()


You: hello nidhi metnal\
Assistant:   Hello Nidhi Metnal,

I hope this message finds you well. If you have any questions or need assistance, feel free to reach out.
You: when can we have sex
Assistant:   I'm sorry, but I cannot provide guidance on personal matters such as sexual activity. It's important to prioritize your health, safety, and the well-being of all parties involved. If you're considering engaging in sexual activity, it's crucial to ensure that you're both consenting adults, have established trust and communication, and are in a safe and comfortable environment. Always remember to practice safe sex and consider the emotional and physical implications of your choices.
You: ok 
Assistant:   If you have any other questions or need information on a different topic, feel free to ask.
You: i want to start a business
Assistant:   Starting a business is an exciting venture that requires careful planning and consideration. Here are some steps to help you get started:

1. Identify 